# Test of the clasification pipeline

In [1]:
import os

# Function to change to the parent directory
def change_to_parent_directory():
    # Check if the directory has already been changed
    if not os.environ.get('DIR_CHANGED'):
        try:
            current_dir = os.path.dirname(os.path.abspath(__file__))
        except NameError:
            current_dir = os.getcwd()
        parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
        os.chdir(parent_dir)
        os.environ['DIR_CHANGED'] = '1'
        print(f"Current working directory changed to: {os.getcwd()}")
    else:
        print("Directory has already been changed.")

# Call the function to change the working directory
change_to_parent_directory()

Current working directory changed to: /home/ihranicky/git/domainradar-clf


## Optional: Create testing dataset
Note: If you want do to this, set create_test_parquet to **True**

In [2]:
create_test_parquet = True

if create_test_parquet:
    import pandas as pd
    import numpy as np

    # List of input Parquet files along with their maximum rows and desired labels
    input_files = [
        {'file': 'testdata/misp_2402.parquet', 'max_rows': 500, 'label': 'phishing'},
        {'file': 'testdata/benign_2312.parquet', 'max_rows': 2000, 'label': 'benign'},
        {'file': 'testdata/malware_bp.parquet', 'max_rows': 500, 'label': 'malware'},
        {'file': 'testdata/dga_2310.parquet', 'max_rows': 500, 'label': 'dga'},
    ]

    # Number of rows to select in total
    n_rows = 1000
    
    # Read the first file to get the initial columns and create the first dataframe
    first_file_info = input_files[0]
    combined_df = pd.read_parquet(first_file_info['file'])
    
    # Limit the number of rows if necessary for the first file
    if len(combined_df) > first_file_info['max_rows']:
        combined_df = combined_df.sample(n=first_file_info['max_rows'], random_state=1)
    
    # Overwrite the "label" column with the specified label for the first file
    combined_df['label'] = first_file_info['label']
    
    # Get the columns from the first dataframe
    all_columns = combined_df.columns.tolist()

    # Process the remaining files
    for file_info in input_files[1:]:
        df = pd.read_parquet(file_info['file'])
        
        # Limit the number of rows if necessary
        if len(df) > file_info['max_rows']:
            df = df.sample(n=file_info['max_rows'], random_state=1)
        
        # Overwrite the "label" column with the specified label
        df['label'] = file_info['label']
        
        # Ensure all columns from the first dataframe are present
        for col in all_columns:
            if col not in df.columns:
                df[col] = None
        
        # Align the dataframe to the columns of the first dataframe
        df = df[all_columns]
        
        # Append the dataframe to the combined dataframe
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    # Randomly select n_rows rows from the combined DataFrame
    selected_rows = combined_df.sample(n=n_rows, random_state=1)  # random_state for reproducibility
    
    # Save the selected rows to a new Parquet file
    selected_rows.to_parquet('testdata/sample.parquet')

/tmp/ipykernel_533632/946921797.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([combined_df, df], ignore_index=True)


## Run classification

In [3]:
# Specify the parquet file with the dataset for classification
test_dataset = 'testdata/sample.parquet'

# Number of domain names to classify with each run of the pipeline (0 = classify all)
CHUNK_SIZE = 30

In [4]:
import pandas as pd
import numpy as np
from classifiers.pipeline import Pipeline

# Initialize the classification pipeline
clf = Pipeline()

# Read the input parquet file
input_df = pd.read_parquet(test_dataset)

# Determine the number of chunks
num_chunks = (len(input_df) + CHUNK_SIZE - 1) // CHUNK_SIZE if CHUNK_SIZE > 0 else 1

# Process the dataframe in chunks
for i in range(num_chunks):
    if CHUNK_SIZE > 0:
        start_idx = i * CHUNK_SIZE
        end_idx = start_idx + CHUNK_SIZE
        chunk_df = input_df[start_idx:end_idx]
    else:
        chunk_df = input_df

    # Perform your classification or processing on the working_df here
    print(f"===== Processing chunk {i+1}/{num_chunks} =====")

    chunk_without_label = chunk_df.drop(columns=['label']) # Label should not be known to classifiers
    chunk_results = clf.classify_domains(chunk_without_label)

    for result in chunk_results:
        print(result)
     
    print(f"===== Chunk {i+1}/{num_chunks} completed. =====")


2024-05-27 17:24:17.008971: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 17:24:17.008996: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 17:24:17.009743: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-27 17:24:17.014302: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 17:24:17.693264: W tensorflow/compiler/tf2

===== Processing chunk 1/34 =====
{'domain': 'cocinortemtb.com.do', 'aggregate_probability': 0.0056352862156927586, 'aggregate_description': '...', 'classification_results': [{'classifier': 'Phishing', 'probability': 0.011665243761273184, 'description': 'No phishing detected.', 'details': {'CNN phishing classifier': '0.0%', 'LightGBM phishing classifier': '2.33%'}}, {'classifier': 'Malware', 'probability': 1.1824990906461608e-06, 'description': 'No malware detected.', 'details:': {'CNN malware classifier': '0.0%', 'XGBoost malware classifier': '0.0%'}}, {'classifier': 'DGA', 'probability': 2.1868197563890135e-06, 'description': 'No DGA detected.', 'details:': {}}]}
{'domain': 'beautysalon.melbourne', 'aggregate_probability': 0.15612971782684326, 'aggregate_description': '...', 'classification_results': [{'classifier': 'Phishing', 'probability': 0.5006298073131508, 'description': 'The domain has high level of phishing indicators.', 'details': {'CNN phishing classifier': '100.0%', 'Light

## Optional: Generate preliminary results for training the final aggregation classifier

In [5]:
import pandas as pd
import numpy as np
from classifiers.pipeline import Pipeline

# Initialize the classification pipeline
clf = Pipeline()

# Read the input parquet file
input_df = pd.read_parquet(test_dataset)

input_df = input_df.sample(frac=1).reset_index(drop=True)

#preliminary_results_df = clf.generate_preliminary_results(input_df, output_file="test_preliminary_results.parquet")
preliminary_results_df = clf.generate_preliminary_results(input_df, add_final=True)

In [6]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)
pd.options.display.float_format = '{:.6f}'.format
preliminary_results_df

,domain_name,dns_available,dns_nonzero,tls_available,tls_nonzero,ip_available,ip_nonzero,rdap_available,rdap_nonzero,geo_available,geo_nonzero,label,phishing_cnn_result,phishing_lgbm_result,malware_cnn_result,malware_xgboost_result,dga_binary_nn_result,phishing_sum,phishing_avg,phishing_prod,malware_sum,malware_avg,malware_prod,total_sum,total_avg,total_prod,badness_probability
0,lgsc.gov.zm,0.900000,0.525000,1.000000,0.458333,1.000000,1.000000,1.000000,0.416667,1.000000,0.944444,benign,0.000000,0.001492,0.000000,0.000062,0.024016,0.001492,0.000746,0.000000,0.000062,0.000031,0.000000,0.025570,0.005114,0.000000,0.003441
1,ehajilwttk.com,0.475000,0.000000,0.041667,0.000000,0.625000,0.000000,0.833333,0.000000,1.000000,0.000000,dga,1.000000,0.010479,0.000000,0.990191,0.989873,1.010479,0.505239,0.010479,0.990191,0.495095,0.000000,2.990543,0.598109,0.000000,0.997001
2,installationsoftware1.com,0.600000,0.125000,0.041667,0.000000,0.625000,0.125000,1.000000,0.291667,1.000000,0.000000,malware,1.000000,0.997641,1.000000,0.999806,0.000003,1.997641,0.998821,0.997641,1.999806,0.999903,0.999806,3.997450,0.799490,0.000003,0.999987
3,waplog.reviews,0.900000,0.550000,1.000000,0.458333,1.000000,0.875000,1.000000,0.833333,1.000000,1.000000,benign,0.000000,0.005807,0.000000,0.000001,0.000003,0.005807,0.002903,0.000000,0.000001,0.000001,0.000000,0.005811,0.001162,0.000000,0.003541
4,1694643313m.donttuchme.gay,0.600000,0.200000,1.000000,0.458333,1.000000,0.625000,0.833333,0.291667,1.000000,0.611111,benign,1.000000,0.000446,1.000000,0.000066,0.000000,1.000446,0.500223,0.000446,1.000066,0.500033,0.000066,2.000512,0.400102,0.000000,0.021153
5,cocinortemtb.com.do,1.000000,0.725000,1.000000,0.458333,1.000000,1.000000,0.833333,0.291667,1.000000,0.722222,benign,0.000000,0.023330,0.000000,0.000002,0.000002,0.023330,0.011665,0.000000,0.000002,0.000001,0.000000,0.023335,0.004667,0.000000,0.005635
6,iuvskjhrpkmhl.web.app,0.600000,0.225000,1.000000,0.458333,1.000000,0.625000,1.000000,0.833333,1.000000,0.722222,phishing,1.000000,0.993262,0.000000,0.804447,0.000033,1.993262,0.996631,0.993262,0.804447,0.402224,0.000000,2.797742,0.559548,0.000000,0.999928
7,cdn.csml.dev,0.600000,0.200000,1.000000,0.833333,1.000000,0.625000,1.000000,0.708333,1.000000,0.722222,benign,0.000000,0.001567,0.000000,0.000092,0.000000,0.001567,0.000783,0.000000,0.000092,0.000046,0.000000,0.001659,0.000332,0.000000,0.008087
8,advair.charity,0.900000,0.525000,0.041667,0.000000,1.000000,1.000000,1.000000,0.833333,1.000000,1.000000,benign,1.000000,0.000385,1.000000,0.000001,0.000029,1.000385,0.500193,0.000385,1.000001,0.500000,0.000001,2.000415,0.400083,0.000000,0.004654
9,credit.j-payment.co.jp,0.600000,0.200000,1.000000,0.666667,1.000000,0.750000,0.958333,0.541667,1.000000,0.666667,benign,0.000000,0.000123,0.000000,0.000017,0.000000,0.000123,0.000061,0.000000,0.000017,0.000008,0.000000,0.000140,0.000028,0.000000,0.005764


In [7]:
preliminary_results_df[["domain_name", "label", "phishing_cnn_result", "phishing_lgbm_result", "malware_cnn_result", "malware_xgboost_result", "dga_binary_nn_result"]]

,domain_name,label,phishing_cnn_result,phishing_lgbm_result,malware_cnn_result,malware_xgboost_result,dga_binary_nn_result
0,lgsc.gov.zm,benign,0.000000,0.001492,0.000000,0.000062,0.024016
1,ehajilwttk.com,dga,1.000000,0.010479,0.000000,0.990191,0.989873
2,installationsoftware1.com,malware,1.000000,0.997641,1.000000,0.999806,0.000003
3,waplog.reviews,benign,0.000000,0.005807,0.000000,0.000001,0.000003
4,1694643313m.donttuchme.gay,benign,1.000000,0.000446,1.000000,0.000066,0.000000
5,cocinortemtb.com.do,benign,0.000000,0.023330,0.000000,0.000002,0.000002
6,iuvskjhrpkmhl.web.app,phishing,1.000000,0.993262,0.000000,0.804447,0.000033
7,cdn.csml.dev,benign,0.000000,0.001567,0.000000,0.000092,0.000000
8,advair.charity,benign,1.000000,0.000385,1.000000,0.000001,0.000029
9,credit.j-payment.co.jp,benign,0.000000,0.000123,0.000000,0.000017,0.000000


In [9]:
preliminary_results_df[["domain_name", "label", "badness_probability"]]

,domain_name,label,badness_probability
0,lgsc.gov.zm,benign,0.003441
1,ehajilwttk.com,dga,0.997001
2,installationsoftware1.com,malware,0.999987
3,waplog.reviews,benign,0.003541
4,1694643313m.donttuchme.gay,benign,0.021153
5,cocinortemtb.com.do,benign,0.005635
6,iuvskjhrpkmhl.web.app,phishing,0.999928
7,cdn.csml.dev,benign,0.008087
8,advair.charity,benign,0.004654
9,credit.j-payment.co.jp,benign,0.005764
